# 0. Import module & libraries

In [1]:
import os
import re
import sys
import pickle
import importlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Specify the directory containing the .py modules
module_dir = "../modules"

# Add the directory to the Python path
sys.path.append(os.path.abspath(module_dir))

In [2]:
from datasets_v2 import KnowledgeBase

# 1. Build PAARAL Knowledge Base
* * *
**Notes**:
* The scope of the data we will use is only until `SY 2023-2024` to match the available data private schools have.
* To build our network, we use the following datasets from the Department of Education (DepEd) for public and private in order of importance:
    * Longitude and Latitude (as of SY 23-24)
    * SY 23-24 Enrollment
    * SY 23-24 Furnitures, namely Seats
    * SY 22-23 Public Shifting Schedule
    * SHS School Offerings
    * ESC & SHS VP (GASTPE) delivering Private Schools
    * GASTPE Top-ups

In [3]:
# Run pipeline that loads and preprocesses raw data from DepEd
kb = KnowledgeBase(load_knowledge_base=True)

Loading public school coordinates as of SY 2023-2024.
Time elapsed for public school coordinates: 6.47 seconds

Loading private school coordinates as of 2024.
Time elapsed for private school coordinates: 5.11 seconds

Loading public and private school enrollment & SHS offerings for SY 2023-2024.
Time elapsed for enrollment & SHS offerings: 6.76 seconds

Loading public & private school furnitures, namely seats, for SY 2023-2024.
Time elapsed for public & private seats: 13.99 seconds

Loading public school shifting schedule for SY 2023-2024.
Time elapsed for public shifting: 44.61 seconds

Loading private school ESC and SHS VP delivering schools as of 2024.
Time elapsed for public shifting: 1.92 seconds



## 1.1. Inspect datasets
Let us view the content of the rows and columns of the datasets we loaded and preprocessed.

In [4]:
# Display dataframe of public school coordinates
display(kb.public_coordinates.sample(3))

,region,division,district,school_id,school_name,street_address,province,municipality,legislative_district,barangay,longitude,latitude
5022,Region II,Quirino,Diffun I,104283,Isidro Paredes ES,-PUROK 1 MAGSAYSAY STREET,QUIRINO,DIFFUN,Lone District,ISIDRO PAREDES,121.512480,16.608160
13404,MIMAROPA,Oriental Mindoro,Bongabong South,110403,Ogbot ES,Ogbot,ORIENTAL MINDORO,BONGABONG,2nd District,OGBOT,121.516373,12.634214
39968,Region XII,Sultan Kudarat,Kulaman II,131071,Buklod Elementary School,Purok 1,SULTAN KUDARAT,SEN. NINOY AQUINO,2nd District,BUKLOD,124.366916,6.369476


In [5]:
# Display dataframe of private school coordinates
display(kb.private_coordinates.sample(3))

,region,division,district,school_id,school name,street address,mother school id,province,municipality,legislative district,barangay,sector,school subclassification,modified coc,latitude,longitude,sheet_name,no.
34,NCR,Muntinlupa City,Muntinlupa City District I,407344,The Wise Kids School of Muntinlupa,"Blk 7 Lot 65-68 Dollar St., Tunasan Muntinlupa...",NaN,NCR FOURTH DISTRICT,CITY OF MUNTINLUPA,Lone District,TUNASAN,Private,Non-Sectarian,ES and JHS,14.369375,121.047506,Muntinlupa City,NaN
175,Region V,Camarines Sur,Tigaon,403751,Partido Christian Academy Foundation Inc.,Zone 2,NaN,CAMARINES SUR,TIGAON,4th District,SAN FRANCISCO,Private,Non-Sectarian,Purely ES,13.626024,123.491413,Camarines Sur,NaN
141,Region IV-A,Quezon,San Narciso,425073,Our Lady of Peace Parochial School of Abuyon (...,NONE,NaN,QUEZON,SAN NARCISO,3rd District,ABUYON,Private,Non-Sectarian,Purely ES,13.62287,122.520146,Quezon,NaN


In [6]:
# Display dataframe of public and private school enrollment
display(kb.enrollment.sample(3))

,school_id,category,count_enrollment,grade_level,sex,shs_strand,level_of_education
231641,132191,Grade 1_Female,5.0,Grade 1,Female,Not Applicable,Elementary
137131,164015,Grade 1_Male,293.0,Grade 1,Male,Not Applicable,Elementary
1623924,305706,Grade 11_Male-ABM,12.0,Grade 11,Male,ABM,Senior High School


In [7]:
# Display dataframe of public and private SHS offerings
display(kb.shs_offerings.sample(3))

shs_strand,ABM,ARTS & DESIGN,GAS,HUMSS,PBM,SPORTS,STEM,TVL
school_id,,,,,,,,
302024,0,0,1,0,0,0,0,0
314901,0,0,0,1,0,0,0,1
301922,1,0,1,1,0,0,1,1


In [8]:
# Display dataframe of public schools shifting schedules (see far right column)
display(kb.public_shifting_info.sample(3))

,region,division,district,school_id,school name,street address,province,municipality,legislative district,barangay,sector,school subclassification,modified curricular offering classification,shifting schedule being implemented
34802,Region X,Misamis Occidental,Lopez Jaena,127422,Macalibre Alto ES,Poblacion Macalibre Alto,MISAMIS OCCIDENTAL,LOPEZ JAENA,1st District,MACALIBRE ALTO,Public,DepED Managed,Purely ES,No Shift
764,Region I,Ilocos Sur,Sta. Lucia,100690,Damacuag Elementary School,nan,ILOCOS SUR,SANTA LUCIA,2nd District,DAMACUAG,Public,DepED Managed,Purely ES,No Shift
42014,CARAGA,Surigao del Sur,San Miguel II,214541,Kiskis Elementary School of Living Tradition,"Kiskis, Siagao, San Miguel, Surigao del Sur",SURIGAO DEL SUR,SAN MIGUEL,1st District,SIAGAO,Public,DepED Managed,Purely ES,No Shift


In [9]:
# Display dataframe of private ESC and SHS VP delivering schools
print(kb.gastpe.shape)
display(kb.gastpe.sample(2))

(5188, 14)


,esc_participating,shsvp_participating,suc/luc,esc_school_id,school_name,esc_(tuition),esc_(other),esc_(misc),esc_(total),shsvp_(tuition),shsvp_(other),shsvp_(misc),shsvp_(total),esc_amount
school_id,,,,,,,,,,,,,,
406408,1,0,0,1402851,St. Claire School Basic Education Inc.,31350.0,0.0,19900.0,51250.0,0.0,0.0,0.0,0.0,13000.0
442619,1,1,0,604351,"Ang Dios Gugma Catholic School, Inc.",20000.0,500.0,5000.0,25500.0,25000.0,4900.0,5000.0,34900.0,11000.0


## 1.2. Compile public datasets

In [10]:
%%time
compiled_public = kb.compile_public_datasets()
print(compiled_public.shape)
display(compiled_public.sample(3))

(47821, 24)


,region,division,school_name,province,municipality,longitude,latitude,shifting_schedule,modified coc,seats_es,...,enrollment_shs,shs_ABM,shs_ARTS & DESIGN,shs_GAS,shs_HUMSS,shs_PBM,shs_SPORTS,shs_STEM,shs_TVL,sector
school_id,,,,,,,,,,,,,,,,,,,,,
219519,CAR,Benguet,Bakian Guiniawan PS,BENGUET,BOKOD,120.878532,16.497938,No Shift,Purely ES,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Public
102483,Region II,Cagayan,Barsat West ES,CAGAYAN,BAGGAO,121.800000,17.900000,No Shift,Purely ES,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Public
300086,Region I,Ilocos Sur,Tagudin National High School,ILOCOS SUR,TAGUDIN,120.443453,16.938608,No Shift,JHS with SHS,NaN,...,1272.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,Public


CPU times: user 609 ms, sys: 50.5 ms, total: 660 ms
Wall time: 659 ms


In [12]:
# Let's display the features/columns of this wide dataframe
display(compiled_public.columns)

Index(['region', 'division', 'school_name', 'province', 'municipality',
       'longitude', 'latitude', 'shifting_schedule', 'modified coc',
       'seats_es', 'seats_jhs', 'seats_shs', 'enrollment_es', 'enrollment_jhs',
       'enrollment_shs', 'shs_ABM', 'shs_ARTS & DESIGN', 'shs_GAS',
       'shs_HUMSS', 'shs_PBM', 'shs_SPORTS', 'shs_STEM', 'shs_TVL', 'sector'],
      dtype='object')

## 1.3. Compile private datasets

In [13]:
%%time
compiled_private = kb.compile_private_datasets()
print(compiled_private.shape)
display(compiled_private.sample(3))

(12113, 38)


,region,division,school_name,province,municipality,barangay,school type,modified coc,enrollment_es,enrollment_jhs,...,longitude,shs_ABM,shs_ARTS & DESIGN,shs_GAS,shs_HUMSS,shs_PBM,shs_SPORTS,shs_STEM,shs_TVL,sector
school_id,,,,,,,,,,,,,,,,,,,,,
404857,Region IX,Zamboanga del Sur,Sacred Heart Kindergarten School,ZAMBOANGA DEL SUR,MOLAVE,MALOLOY-ON (POB.),School with no Annexes,All Offering,28.0,528.0,...,123.489154,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,Private
400242,Region I,"Pangasinan II, Binalonan","St. Patrick Catholic School, Inc.",PANGASINAN,TAYUG,BARANGAY A (POB.),School with no Annexes,Purely ES,97.0,NaN,...,120.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private
408554,Region V,Camarines Sur,"Sunrise Technological School (STS), Inc.",CAMARINES SUR,TIGAON,MAY-ANAO,School with no Annexes,Purely ES,6.0,NaN,...,123.494443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private


CPU times: user 497 ms, sys: 19.9 ms, total: 517 ms
Wall time: 516 ms


In [14]:
# Let's display the features/columns of this wide dataframe
display(compiled_private.columns)

Index(['region', 'division', 'school_name', 'province', 'municipality',
       'barangay', 'school type', 'modified coc', 'enrollment_es',
       'enrollment_jhs', 'enrollment_shs', 'seats_kinder', 'seats_es',
       'seats_jhs', 'seats_shs', 'esc_participating', 'shsvp_participating',
       'esc_school_id', 'esc_(tuition)', 'esc_(other)', 'esc_(misc)',
       'esc_(total)', 'shsvp_(tuition)', 'shsvp_(other)', 'shsvp_(misc)',
       'shsvp_(total)', 'esc_amount', 'latitude', 'longitude', 'shs_ABM',
       'shs_ARTS & DESIGN', 'shs_GAS', 'shs_HUMSS', 'shs_PBM', 'shs_SPORTS',
       'shs_STEM', 'shs_TVL', 'sector'],
      dtype='object')

# 2. Save knolwedge base
* * *
So that we will not have to rerun the pipeline, we generate local files of our preprocessed and compiled datasets for public and private schools. Again, we note that the scope of the information contained in these datasets is for school year 2023-2024 only.

In [15]:
!ls '../datasets'

 calabarzon_polillo_etal.geojson
 mimaropa_overpass_roads.geojson
 mimaropa_overpass_RoadsWalks.geojson
 networks
 philippines-json-maps
 philippines-psgc-maps
 private
 processed
 public
'_SY 2024-2025 SCHOOL LEVEL DATA ON ENROLLMENT.xlsx'


In [16]:
# Checks if local has a "datasets" directory exists
dirpath = '../datasets'

if not os.path.exists(dirpath):
    os.makedirs(dirpath)
    print(f"Created directory with path {dirpath}")

    dirpath_processed = os.path.join(dirpath, 'processed/')
else:
    dirpath_processed = os.path.join(dirpath, 'processed/')

files = [
    compiled_public,
    compiled_private
]
filenames = [
    'compiled_public_datasets.csv',
    'compiled_private_datasets.csv'
]
# Checks if local has a "datasets/processed" directory exists
if os.path.exists(dirpath_processed):
    for i, filename in enumerate(filenames):
        save_path = os.path.join(dirpath_processed, filename)
    
        # Save compiled datasets as a CSV
        files[i].to_csv(save_path, index=True)
        print(f"Saved compiled datasets with path {save_path}")
else:
    # Create datasets/processed directory if it does NOT exist
    os.makedirs(dirpath_processed)
    print(f"Created directory with path {dirpath_processed}")

    for i, filename in enumerate(filenames):
        save_path = os.path.join(dirpath_processed, filename)
    
        # Save compiled datasets as a CSV
        files[i].to_csv(save_path, index=True)
        print(f"Saved compiled datasets with path {save_path}")

Saved compiled datasets with path ../datasets/processed/compiled_public_datasets.csv
Saved compiled datasets with path ../datasets/processed/compiled_private_datasets.csv


In [17]:
# Let's see if we're successful in saving our compiled datasets by loading them
comp_pub = pd.read_csv('../datasets/processed/compiled_public_datasets.csv')
print(comp_pub.shape)
display(comp_pub.head(2))

print()

comp_priv = pd.read_csv('../datasets/processed/compiled_private_datasets.csv')
print(comp_priv.shape)
display(comp_priv.head(2))

(47821, 25)


,school_id,region,division,school_name,province,municipality,longitude,latitude,shifting_schedule,modified coc,...,enrollment_shs,shs_ABM,shs_ARTS & DESIGN,shs_GAS,shs_HUMSS,shs_PBM,shs_SPORTS,shs_STEM,shs_TVL,sector
0,100001,Region I,Ilocos Norte,Apaleng-Libtong ES,ILOCOS NORTE,BACARRA,120.614372,18.266860,No Shift,Purely ES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Public
1,100002,Region I,Ilocos Norte,Bacarra CES,ILOCOS NORTE,BACARRA,120.609487,18.251272,No Shift,Purely ES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Public



(12113, 39)


,school_id,region,division,school_name,province,municipality,barangay,school type,modified coc,enrollment_es,...,longitude,shs_ABM,shs_ARTS & DESIGN,shs_GAS,shs_HUMSS,shs_PBM,shs_SPORTS,shs_STEM,shs_TVL,sector
0,400001,Region I,Ilocos Norte,"St. Andrew Academy of Bacarra, Inc.",ILOCOS NORTE,BACARRA,SANTA RITA (POB.),School with no Annexes,JHS with SHS,NaN,...,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Private
1,410001,Region I,Ilocos Norte,"The Riverdeep Academy, Inc.",ILOCOS NORTE,BACARRA,CABUSLIGAN,School with no Annexes,Purely ES,235.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private


In [18]:
comp_priv.columns

Index(['school_id', 'region', 'division', 'school_name', 'province',
       'municipality', 'barangay', 'school type', 'modified coc',
       'enrollment_es', 'enrollment_jhs', 'enrollment_shs', 'seats_kinder',
       'seats_es', 'seats_jhs', 'seats_shs', 'esc_participating',
       'shsvp_participating', 'esc_school_id', 'esc_(tuition)', 'esc_(other)',
       'esc_(misc)', 'esc_(total)', 'shsvp_(tuition)', 'shsvp_(other)',
       'shsvp_(misc)', 'shsvp_(total)', 'esc_amount', 'latitude', 'longitude',
       'shs_ABM', 'shs_ARTS & DESIGN', 'shs_GAS', 'shs_HUMSS', 'shs_PBM',
       'shs_SPORTS', 'shs_STEM', 'shs_TVL', 'sector'],
      dtype='object')